# Generador datos sintéticos para NER

Los datos sintéticos generados se guardan en formato json.

Posteriormente se hace un revisión rápida de los datos utilizando label studio

En label studio se guardan los archivos con extension .conll, el cual es utilizado por el modelo

In [ ]:
!pip install pandas openai tqdm

In [ ]:
#Configurar API de DeepSeek con sintaxis actualizada
from openai import OpenAI
import pandas as pd
import json

client = OpenAI(
    api_key="<API-PROPIA>",
    base_url="https://api.deepseek.com/v1"
)

def callPromptLLM():
    try:
        response = client.chat.completions.create(
            model="deepseek-reasoner",
            messages=[
                {"role": "system", "content":"""
               Eres un servicio API de generación de datos. Tu única función es devolver un objeto JSON válido, bien formado y directamente parseable.
                """},
                {"role": "user", "content": """
Genera un (2) textos (representados por objetos) que simule ser extractos de resoluciones o documentos oficiales de una institución de educación superior. Debe ser único, formal, coherente y no debe superar las 500 palabras.
Reglas Críticas:
Contenido Obligatorio: El texto debe contener, de forma natural, una o dos instancias de CADA UNA de las siguientes entidades nombradas.
Definición de Entidades a Incluir en CADA Texto:
  *nombres de personas:
    Formato: Nombres (1 o 2 nombres + 1 o 2 apellidos) o un solo nombre en un contexto formal.
    Ejemplos: Alvaro Andrés Tituana Valarezo, Melissa Aguilar, Oscar Ignacio Rocha, Pedro.
    Variación: Usa diferentes nombres y apellidos, tanto comunes como no comunes en el contexto hispanohablante.
  *cargos:
    Formato: Puestos de trabajo o roles dentro de una institución.
    Ejemplos: Rector, Secretaria, Vicerrectora Académica, Coordinador del Nivel de apoyo de la Unidad de Relaciones Públicas y Comunicación Social.
    Variación: Crea cargos realistas, desde los más simples hasta los más complejos y específicos.
  *nombres de proyectos:
    Formato: Nombres de planes, programas o iniciativas institucionales.
    Ejemplos: Proyectos de Vinculación 2023-2025, Programa de Certificaciones Internacionales en TI, Iniciativa Verde Campus Limpio, Iniciativa de Transformación Digital.
    Variación: Inventa nombres de proyectos creíbles, combinando áreas (tecnología, vinculación, bienestar, etc.) con años o acrónimos.
  *fechas:
    Formato: Debe ser muy variado.
    Ejemplos: 07 de septiembre de 2022; 6 de noviembre; seis (6) días del mes de noviembre de 2023; 1 de enero de 2024.
    Variación: Alterna entre formatos numéricos, alfanuméricos y texto completo. Usa diferentes días, meses y años.
  *montos de dinero:
    Formato: Cantidades numéricas en dólares, con formatos diversos.
    Ejemplos: 67.56$; 83,5$;$ 12,5;$ 5,234; 45 dolares; 3.65 dolares.
    Variación: Usa puntos y comas como separadores decimales, coloca el símbolo '$' antes o después, y a veces usa la palabra "dolares".
  *referencias a artículos de reglamento:
    Formato: Menciones a artículos de normativas específicas. El número del artículo debe ser aleatorio.
    Ejemplos: el artículo 83 de la Ley Orgánica de Educación Superior; el artículo 15 de la Constitución de la República del Ecuador; el artículo 22 del Estatuto del Instituto Superior Tecnológico Yaruquí.
    Variación: Cambia el número del artículo y la ley referenciada en cada texto.
Diversificación Imperativa: Es fundamental que varíes la estructura, el contenido y las palabras iniciales de cada entidad generada. No uses siempre los mismos ejemplos. El objetivo es que el modelo NER aprenda patrones variados y no memorice instancias específicas.
Formato de Salida Esperado:
  *La salida debe ser un único arreglo (lista) que contenga los dos objetos generados. Cada objeto debe seguir estrictamente la siguiente estructura:
  {
    "data":{
      "text": Una cadena de texto con el párrafo completo del documento sintético.
      },
    "annotations": [
        {
          "result":[
            {
              "from_name": "label",
              "to_name": "text",
              "type": "labels",
              "value": {
                "text": <texto de la entidad>,
                "labels": <etiqueta de la entidad>
              }
            },...
          ]
          ]
        }
      ]
    }
  }
    "text": Una cadena de texto con el párrafo completo del documento sintético.
    "annotations": Una lista de objetos, donde cada objeto representa una entidad nombrada encontrada en el texto.
    Cada objeto de anotación debe contener:
    "entity": La etiqueta de la entidad en mayúsculas (PERSONA, CARGO, PROYECTO, FECHA, DINERO, REGLAMENTO).
    "entity_text": El texto exacto de la entidad extraído del campo "text". Es crítico que esta cadena coincida perfectamente.
  *Ejemplo de la estructura para UN objeto de datos sintéticos en formato JSON:
   [
    {
      "data": {
        "text": "Mediante Acuerdo Nro. 2024-087, emitido el quince de marzo de 2024, se designa a la Ing. Pazmiño en el cargo de Directora del Departamento de Investigación y Desarrollo... "
      },
      "annotations": [
        {
          "result": [
            {
              "from_name": "label",
              "to_name": "text",
              "type": "labels",
              "value": {
                "text": "quince de marzo de 2024",
                "labels": ["FECHA"]
              }
            },
            {
              "from_name": "label",
              "to_name": "text",
              "type": "labels",
              "value": {
                "text": "Pazmiño",
                "labels": ["PERSONA"]
              }
            },
            {
              "from_name": "label",
              "to_name": "text",
              "type": "labels",
              "value": {
                "text": "Directora del Departamento de Investigación y Desarrollo",
                "labels": ["CARGO"]
              }
            }...
          ]
        }
      ]
    }
  ]
  Reglas Críticas:
1.  La salida debe ser **exclusivamente** el código JSON. No incluyas texto, explicaciones o markdown (` ```json `) alrededor del JSON.
2.  Tu respuesta debe comenzar con `[` y terminar con `]`.
3.  Valida la sintaxis del JSON antes de finalizar la respuesta para asegurar que no hay errores de comas, comillas o paréntesis.
                """}
            ],
            max_tokens=5000,
            temperature=0.5,
            top_p=0.9,
            frequency_penalty=0.3,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error procesando")
        return None

In [ ]:
def find_entity_position(text, entity_text):
    """
    Encuentra la posición de inicio y fin de entity_text dentro de text.

    Args:
        text (str): El texto completo donde buscar.
        entity_text (str): El texto a buscar dentro de text.

    Returns:
        tuple: Una tupla (start, end) con las posiciones de inicio y fin.
               Retorna (0, 0) si entity_text no se encuentra en text.
    """
    start = text.find(entity_text)
    if start != -1:
        end = start + len(entity_text)
        return (start, end)
    else:
        return (0, 0)

In [ ]:
def generateSyntheticData(thread_id=None):
  "Siempre devuelve un arreglo con dos registros"
  #Solicitar datos API
  json_string_data = callPromptLLM()

  #Convertir en JSON
  if json_string_data:
      try:
          python_data = json.loads(json_string_data)
          #Añado las posiciones de las entidades, ya que la IA, no es muy buena con eso
          for doc in python_data:
              text = doc['data']['text']
              for annotation_group in doc['annotations']:
                  for annotation in annotation_group['result']:
                      entity_text = annotation['value']['text']
                      start, end = find_entity_position(text, entity_text)
                      annotation['value']['start'] = start
                      annotation['value']['end'] = end
          return python_data
      except json.JSONDecodeError as e:
          print(f"Error decoding JSON: {e}")
          print(json_string_data)
  else:
      print("Failed to generate synthetic data.")


In [ ]:
import concurrent.futures
from typing import List
from tqdm import tqdm

all_results = []
num_threads = 50

with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Crear tareas
    futures = [executor.submit(generateSyntheticData, i) for i in range(num_threads)]

    # Monitorear progreso con barra
    with tqdm(total=num_threads, desc="Procesando hilos") as pbar:
        for future in concurrent.futures.as_completed(futures):
            try:
                result = future.result()
                if result is not None:
                  all_results.append(result[0])
                pbar.update(1)
            except Exception as e:
                print(f"\nError: {e}")
print(f"\nProceso completado. Total resultados: {len(all_results)}")

Procesando hilos:  76%|███████▌  | 38/50 [03:17<00:26,  2.24s/it]

Error decoding JSON: Unterminated string starting at: line 69 column 15 (char 2014)
[
  {
    "data": {
      "text": "El Consejo Universitario, en sesión del 5 de abril de 2024, designó al Ing. Carlos Felipe Méndez Ortega como Director del Departamento de Vinculación con la Sociedad, asignando un presupuesto de 10,000 dólares para la implementación del Proyecto 'Campus Sostenible 2025', en concordancia con lo establecido en el artículo 67 de la Ley de Educación Superior."
    },
    "annotations": [
      {
        "result": [
          {
            "from_name": "label",
            "to_name": "text",
            "type": "labels",
            "value": {
              "text": "5 de abril de 2024",
              "labels": ["FECHA"]
            }
          },
          {
            "from_name": "label",
            "to_name": "text",
            "type": "labels",
            "value": {
              "text": "Carlos Felipe Méndez Ortega",
              "labels": ["PERSONA"]
            

Procesando hilos:  78%|███████▊  | 39/50 [03:17<00:18,  1.65s/it]

Error decoding JSON: Unterminated string starting at: line 40 column 13 (char 1341)
[
  {
    "data": {
      "text": "Mediante Resolución Nro. 2024-056, emitida el diez de mayo de 2024, se nombra a la Dra. Ana Lucía Fernández García como Directora del Programa de Internacionalización Académica. Este nombramiento se realiza en el marco del Proyecto 'Expansión Global 2025', con un presupuesto asignado de 25,000$. Dicha designación está avalada por el artículo 33 del Estatuto Universitario."
    },
    "annotations": [
      {
        "result": [
          {
            "from_name": "label",
            "to_name": "text",
            "type": "labels",
            "value": {
              "text": "diez de mayo de 2024",
              "labels": ["FECHA"]
            }
          },
          {
            "from_name": "label",
            "to_name": "text",
            "type": "labels",
            "value": {
              "text": "Ana Lucía Fernández García",
              "labels": ["PERS

Procesando hilos:  80%|████████  | 40/50 [03:20<00:19,  1.98s/it]

...

Error decoding JSON: Expecting property name enclosed in double quotes: line 96 column 23 (char 3117)
[
  {
    "data": {
      "text": "Mediante Acuerdo Nro. 2024-067, emitido el 3 de agosto de 2024, se designa a la Ing. Sofia Catalina Ríos González en el cargo de Vicerrectora de Investigación y Posgrado, para liderar la implementación del Plan de Fortalecimiento Académico 2024-2025, con una asignación presupuestaria de 50,000$. Esta decisión se basa en lo dispuesto en el artículo 27 del Reglamento de Gestión Académica."
    },
    "annotations": [
      {
        "result": [
          {
            "from_name": "label",
            "to_name": "text",
            "type": "labels",
            "value": {
              "text": "3 de agosto de 2024",
              "labels": ["FECHA"]
            }
          },
          {
            "from_name": "label",
            "to_name": "text",
            "type": "labels",
            "value": {
              "text": "Sofia Catalina Ríos Gonzále

Procesando hilos:  98%|█████████▊| 49/50 [03:27<00:01,  1.23s/it]

Failed to generate synthetic data.


Procesando hilos: 100%|██████████| 50/50 [04:24<00:00,  5.28s/it]


Proceso completado. Total resultados: 39


In [ ]:
len(all_results)

39

In [ ]:
import json

output_filename = 'output_data.json'

with open(output_filename, 'w') as f:
    json.dump(all_results, f, indent=4)

print(f"Datos guardados exitosamente en '{output_filename}'")

Datos guardados exitosamente en 'output_data.json'
